# Évaluation de la classification des textes
Cette évaluation ressemble beaucoup au projet de classification de texte que nous venons de terminer, et le jeu de données est très similaire.

Le jeu de données **moviereviews2.tsv** contient le texte de 6000 critiques de films. 3000 sont positives, 3000 sont négatives, et le texte a été prétraité sous la forme d'un fichier délimité par des tabulations. Comme précédemment, les étiquettes sont données comme `pos` et `neg`.

Nous avons inclus 20 critiques qui contiennent soit des données `NaN`, soit des chaînes composées d'espaces blancs.

Pour plus d'informations sur cet ensemble de données, visitez http://ai.stanford.edu/~amaas/data/sentiment/

### Tâche #1 : Effectuer les importations et charger le jeu de données dans un DataFrame pandas
Pour cet exercice, vous pouvez charger le jeu de données à partir de `'./moviereviews2.tsv'`.

In [48]:
import numpy as np
import pandas as pd

df = pd.read_csv('./moviereviews2.tsv', sep='\t')
df.head()

,label,review
0,pos,I loved this movie and will watch it again. Or...
1,pos,"A warm, touching movie that has a fantasy-like..."
2,pos,I was not expecting the powerful filmmaking ex...
3,neg,"This so-called ""documentary"" tries to tell tha..."
4,pos,This show has been my escape from reality for ...


In [49]:
len(df)

6000

### Tâche #2 : Vérifier les valeurs manquantes :

In [50]:
# Check for NaN values:
df.isnull().sum()

label      0
review    20
dtype: int64

In [51]:
# Check for whitespace strings (it's OK if there aren't any!):

blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

0 blanks:  []


### Tâche #3 : Supprimer les valeurs NaN :

In [52]:
df.dropna(inplace=True)

len(df)

5980

### Tâche #4 : Jetez un coup d'œil rapide à la colonne `label` :

In [53]:
df['label'].unique()

array(['pos', 'neg'], dtype=object)

In [54]:
df['label'].value_counts()

neg    2990
pos    2990
Name: label, dtype: int64

### Tâche #5 : Diviser les données en ensembles d'entrainement et de test :
Vous pouvez utiliser les paramètres de votre choix. Pour comparer vos résultats à ma resolution, utilisez `test_size=0.33, random_state=42`

In [55]:
X = df['review']
y = df['label']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

Training Data Shape: (4006,)
Testing Data Shape:  (1974,)


### Tâche #6 : Construire un pipeline pour vectoriser la date, puis entraîner et ajuster un modèle
Vous pouvez utiliser le modèle de votre choix. Pour comparer vos résultats à ma résolution, utilisez `LinearSVC`.

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)

from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])


text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

### Tâche #7 : Exécuter les prédictions et analyser les résultats

In [61]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [62]:
# Report the confusion matrix

from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[900  91]
 [ 63 920]]


In [63]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.93      0.91      0.92       991
         pos       0.91      0.94      0.92       983

    accuracy                           0.92      1974
   macro avg       0.92      0.92      0.92      1974
weighted avg       0.92      0.92      0.92      1974



In [64]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.9219858156028369
